In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import corrections as cor

In [4]:
#Define x bins
x_edges = np.arange(0,1.01,0.03)
x_centers = [x+0.015 for x in x_edges]

#String of kins used for finding the runlists
kins = ['0','1','2','3','4','5','7_1st','7_2nd','9_1st','9_2nd','11_1st','11_2nd','13_1st','13_2nd','15_1st','15_2nd','15_3rd','16_1st','16_2nd']
targs = ['H3','He3','H2','H1']

#Run Metadata
meta = pd.read_csv("ev_clean/meta.csv",index_col='run')

In [ ]:
for targ in targs:
    for i,kin in enumerate(kins):
        #Kinematic charge
        kinQ = 0
        
        #Kinematic Boiling Error
        kinBoilE = 0
        
        #Yield Dataframe
        df = pd.DataFrame(columns=['Yield','Counts','RelStat','RelSyst','Avgx','AvgQ2'])
        
        #Initialize values
        df['Yield'].values[:] = 0
        df['Counts'].values[:] = 0
        df['RelStat'].values[:] = 0
        df['RelSyst'].values[:] = 0
        df['xSum'].values[:] = 0
        df['Q2Sum'].values[:] = 0
        
        try:
            with open("./runlists/" + targ + "_kin" + kin + ".dat","r") as f:
                runs = f.read().splitlines()
                
                for run in runs:
                    #Load run
                    ev = pd.read_csv("ev_clean/" + run + ".csv")
                    
                    #Add in this runs charge
                    kinQ = kinQ + meta.at[int(run),'Q']
                    
                    #Bin in x
                    ev['x_bin'] = np.digitize(ev.x,x_edges)
                    
                    for j in range(len(x_centers)):
                        tmp = ev.query('x_bin==' + str(j+1))
                        df.Counts.iat[j] = tmp.shape[0]
                        df.xSum.iat[j] = tmp.x.sum
                        df.Q2Sum.iat[j] = tmp.Q2.sum
                        
                    df.RelStat = df.Counts.apply(lambda x: 1./math.sqrt(x))
                    
                    #Apply livetime
                    df.Yield = df.Counts.apply(lambda x: x/meta.at[int(run),'lt'])
                    
                    #Divide by scattering centers
                    sc = getattr(cor,targ + 'Nucleons')()
                    df.Yield = df.Yield.apply(lambda x: x/sc(meta.at[int(run),'avgI']))
                    
        except FileNotFoundError:
            print ("./runlists/" + targ + "_kin"+kin+".dat" + " File doesn't exist. Skipping.")